# Day19_2: AI 에이전트 (Agent) 구축하기

## 학습 목표

**Part 1: 기초**
1. 표준 LLM 및 RAG 시스템과 AI 에이전트의 차이점 이해하기
2. ReAct (Reason + Act) 프레임워크 개념 이해하기
3. 에이전트의 핵심 구성 요소(LLM, 도구, 프롬프트) 이해하기
4. LangChain @tool 데코레이터로 도구 만들기
5. 에이전트 실행기(Agent Executor) 이해하기

**Part 2: 심화**
1. 여러 도구를 활용하는 멀티-툴 에이전트 구축하기
2. 대화 메모리(Memory) 통합하기
3. RAG 도구 + 웹 검색 도구 결합하기
4. 뉴스 분석가 에이전트 실습하기

---

## 왜 이것을 배우나요?

| 개념 | 실무 활용 | 예시 |
|------|----------|------|
| AI 에이전트 | 자율적인 작업 수행 AI | 일정 관리, 데이터 분석 자동화 |
| ReAct 프레임워크 | 단계적 문제 해결 | 복잡한 질문 분해 및 해결 |
| 도구(Tools) | 외부 시스템 연동 | API 호출, DB 조회, 웹 검색 |
| 메모리 | 대화 맥락 유지 | 멀티턴 챗봇, 상담 봇 |

**분석가 관점**: RAG가 LLM에게 '지식'을 확장해준다면, 에이전트는 LLM의 '행동'을 확장합니다. "오늘 서울 날씨 알려줘", "A사와 B사 주가 비교해줘"와 같이 외부 도구를 사용해야 하는 요청을 자율적으로 처리할 수 있습니다. 이는 진정한 의미의 AI 비서를 만드는 핵심 기술입니다!

---

# Part 1: 기초

---

## 1.1 AI 에이전트란?

### 표준 LLM vs RAG vs 에이전트

지금까지 배운 LLM과 RAG는 "질문 -> 답변" 형태로 동작합니다. 하지만 현실에서는 더 복잡한 작업이 필요합니다.

| 구분 | 작동 방식 | 예시 |
|------|----------|------|
| **표준 LLM** | 질문 -> (기억에서 답변) -> 답변 | "파이썬이 뭐야?" |
| **RAG** | 질문 -> (문서 검색) -> 답변 | "우리 회사 휴가 정책 알려줘" |
| **에이전트** | 질문 -> (생각 -> 행동 -> 관찰)* -> 답변 | "오늘 날씨 알려주고 캘린더에 일정 추가해줘" |

### AI 에이전트의 정의

```
AI 에이전트 = LLM(두뇌) + 도구(손과 발) + 추론(생각)

LLM이 단순히 텍스트를 생성하는 것을 넘어,
외부 세계와 상호작용하며 주어진 목표를 달성하기 위해
자율적으로 행동하는 프로그램입니다.
```

In [ ]:
# 에이전트가 할 수 있는 작업 예시
agent_capabilities = {
    "실시간 정보 조회": {
        "질문": "오늘 서울 날씨 어때?",
        "동작": "날씨 API 호출 -> 정보 추출 -> 답변 생성"
    },
    "데이터 분석": {
        "질문": "A사와 B사의 최근 주가 비교해줘",
        "동작": "금융 API 호출 -> 데이터 수집 -> 비교 분석 -> 시각화"
    },
    "작업 자동화": {
        "질문": "내일 오전 10시에 팀 회의 일정 추가해줘",
        "동작": "캘린더 API 호출 -> 일정 생성 -> 확인 메시지"
    },
    "복합 작업": {
        "질문": "오늘 날씨 좋으면 야외 미팅 일정 잡아줘",
        "동작": "날씨 확인 -> 조건 판단 -> 캘린더 API 호출"
    }
}

print("AI 에이전트가 할 수 있는 작업:")
print("=" * 60)
for capability, details in agent_capabilities.items():
    print(f"\n{capability}")
    print(f"  질문: {details['질문']}")
    print(f"  동작: {details['동작']}")

---

## 1.2 ReAct 프레임워크

### ReAct란?

> **ReAct = Reason (추론) + Act (행동)**
>
> 논문: "ReAct: Synergizing Reasoning and Acting in Language Models" (2022)

AI 에이전트의 핵심 동작 원리입니다. LLM이 **생각(Thought)** 하고, **행동(Action)** 하며, 그 **결과(Observation)** 를 관찰하여 다음 단계를 결정합니다.

### ReAct 사이클

```
[1] Thought (생각)
    - "사용자가 날씨를 물어봤네. 날씨 API를 호출해야겠다."
    
[2] Action (행동)
    - get_weather(city="서울") 도구 호출
    
[3] Observation (관찰)
    - "서울: 맑음, 기온 15도, 미세먼지 좋음"
    
[4] Repeat 또는 Final Answer
    - 정보가 충분하면 -> 최종 답변 생성
    - 정보가 부족하면 -> [1]로 돌아가서 추가 행동
```

In [ ]:
# ReAct 사이클 시뮬레이션
react_example = """
=== ReAct 사이클 예시 ===

사용자 질문: "오늘 서울 날씨가 좋으면 한강 피크닉 어때?"

[Round 1]
Thought: 서울 날씨를 먼저 확인해야 해. get_weather 도구를 사용하자.
Action: get_weather(city="서울")
Observation: 서울 - 맑음, 18도, 미세먼지 좋음

[Round 2]
Thought: 날씨가 좋네! 이제 한강 근처 추천 장소를 검색해보자.
Action: web_search(query="한강 피크닉 추천 장소 2025")
Observation: 1. 여의도 한강공원 2. 반포 한강공원 3. 뚝섬 한강공원...

[Round 3]
Thought: 충분한 정보를 얻었어. 최종 답변을 만들자.
Final Answer: 오늘 서울은 맑고 18도로 피크닉하기 딱 좋은 날씨예요! 
              추천 장소로는 여의도 한강공원, 반포 한강공원이 있습니다.
"""

print(react_example)

### 왜 ReAct가 효과적인가?

| 특징 | 설명 |
|------|------|
| **단계적 추론** | 복잡한 문제를 작은 단계로 분해 |
| **자기 수정** | 중간 결과를 보고 전략 수정 가능 |
| **투명성** | 추론 과정이 명시적으로 보임 |
| **유연성** | 다양한 도구를 상황에 맞게 선택 |

---

## 1.3 에이전트의 핵심 구성 요소

### 3가지 핵심 요소

```
AI 에이전트 = LLM(두뇌) + Tools(도구) + Prompt(지침)
              + Agent Executor(조율자)
```

| 구성 요소 | 역할 | 비유 |
|----------|------|------|
| **LLM** | 추론과 결정 | 두뇌 |
| **Tools** | 외부 세계와 상호작용 | 손과 발 |
| **Prompt** | 행동 지침 제공 | 업무 매뉴얼 |
| **Agent Executor** | 전체 사이클 관리 | 매니저 |

In [ ]:
# 필수 라이브러리 설치
# !pip install langchain langchain-openai python-dotenv

In [ ]:
import os
from dotenv import load_dotenv

# .env 파일에서 API 키 로드
load_dotenv()

# API 키 확인
api_key = os.getenv("OPENAI_API_KEY")
if api_key:
    print(f"OpenAI API Key loaded: {api_key[:8]}...{api_key[-4:]}")
else:
    print("Warning: OPENAI_API_KEY not found!")

### 1.3.1 LLM - 에이전트의 두뇌

In [ ]:
from langchain_openai import ChatOpenAI

# LLM 초기화
# temperature=0: 일관된 답변 (에이전트에 권장)
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

print("LLM (에이전트의 두뇌) 준비 완료!")
print(f"모델: {llm.model_name}")
print(f"Temperature: {llm.temperature}")

---

## 1.4 도구(Tools) 만들기

### 도구란?

에이전트가 외부 세계와 상호작용할 수 있게 해주는 함수입니다.

```
도구 = 함수 + 설명(description)

핵심: LLM은 도구의 '설명'을 읽고 언제 사용할지 결정합니다!
     -> 좋은 설명 = 좋은 도구 선택
```

### @tool 데코레이터

LangChain의 `@tool` 데코레이터를 사용하면 파이썬 함수를 쉽게 도구로 변환할 수 있습니다.

In [ ]:
from langchain.agents import tool
import datetime

@tool
def get_current_time(ignored_input: str = "") -> str:
    """
    현재 한국 시간을 'YYYY-MM-DD HH:MM:SS' 형식으로 반환합니다.
    현재 시각, 지금 몇 시, 오늘 날짜 등의 질문에 사용하세요.
    """
    return datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")

# 도구 정보 확인
print(f"도구 이름: {get_current_time.name}")
print(f"도구 설명: {get_current_time.description}")
print(f"\n테스트 실행: {get_current_time.invoke('')}")

In [ ]:
# 추가 도구 만들기: 간단한 계산기
@tool
def calculator(expression: str) -> str:
    """
    수학 계산을 수행합니다. 덧셈, 뺄셈, 곱셈, 나눗셈을 지원합니다.
    입력: 계산식 문자열 (예: "2 + 3 * 4")
    출력: 계산 결과
    """
    try:
        # 안전한 수학 연산만 허용
        allowed_chars = set('0123456789+-*/().  ')
        if not all(c in allowed_chars for c in expression):
            return "오류: 허용되지 않는 문자가 포함되어 있습니다."
        result = eval(expression)
        return f"{expression} = {result}"
    except Exception as e:
        return f"계산 오류: {str(e)}"

# 테스트
print(f"도구 이름: {calculator.name}")
print(f"테스트: {calculator.invoke('2 + 3 * 4')}")

### 도구 설명 작성 팁

```
좋은 도구 설명:
1. 도구가 무엇을 하는지 명확히 설명
2. 어떤 상황에서 사용해야 하는지 명시
3. 입력과 출력 형식 안내

예:
"현재 한국 시간을 반환합니다. '지금 몇 시야?', '오늘 날짜' 질문에 사용하세요."
```

---

## 1.5 에이전트 실행기 (Agent Executor)

### Agent Executor의 역할

LLM, 도구, 프롬프트를 하나로 묶어 ReAct 사이클을 실행하는 조율자입니다.

```
Agent Executor:
1. 사용자 입력 받기
2. LLM에게 생각(Thought) 요청
3. LLM이 선택한 도구 실행
4. 결과(Observation)를 LLM에게 전달
5. 최종 답변까지 반복
```

In [ ]:
from langchain.agents import create_openai_tools_agent, AgentExecutor
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

# 도구 리스트
tools = [get_current_time, calculator]

# 에이전트 프롬프트 템플릿
prompt = ChatPromptTemplate.from_messages([
    ("system", "당신은 주어진 도구를 활용하여 사용자의 질문에 답변하는 유능한 AI 어시스턴트입니다. 한국어로 친절하게 답변하세요."),
    MessagesPlaceholder(variable_name="chat_history", optional=True),
    ("human", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad"),  # 중간 추론 과정
])

# 에이전트 생성
agent = create_openai_tools_agent(llm, tools, prompt)

# 에이전트 실행기 생성
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True  # 추론 과정 출력
)

print("에이전트 준비 완료!")
print(f"사용 가능한 도구: {[t.name for t in tools]}")

In [ ]:
# 에이전트 테스트 1: 현재 시간 질문
response = agent_executor.invoke({"input": "지금 몇 시야?"})

print("\n" + "=" * 50)
print("최종 답변:")
print(response["output"])

In [ ]:
# 에이전트 테스트 2: 계산 질문
response = agent_executor.invoke({"input": "123 곱하기 456은 얼마야?"})

print("\n" + "=" * 50)
print("최종 답변:")
print(response["output"])

In [ ]:
# 에이전트 테스트 3: 복합 질문 (여러 도구 사용)
response = agent_executor.invoke({"input": "지금 시간 알려주고, 2025에서 현재 년도를 빼면 얼마야?"})

print("\n" + "=" * 50)
print("최종 답변:")
print(response["output"])

### verbose=True 출력 이해하기

```
> Entering new AgentExecutor chain...
Invoking: `get_current_time` with `{}`       <- Action (도구 호출)
2025-01-29 14:30:00                           <- Observation (결과)
현재 시간은 2025년 1월 29일 오후 2시 30분입니다. <- Final Answer

> Finished chain.
```

---

# Part 2: 심화

---

## 2.1 멀티-툴 에이전트 구축

### 목표

여러 도구를 가진 에이전트를 만들어, 상황에 따라 적절한 도구를 선택하도록 합니다.

```
우리가 만들 에이전트:
1. RAG 검색 도구: 뉴스 기사 기반 Q&A
2. 웹 검색 도구: 실시간 정보 검색
3. 시간 도구: 현재 시간 조회
```

In [ ]:
# 필요한 라이브러리
# !pip install langchain-chroma chromadb duckduckgo-search

In [ ]:
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma
from langchain.docstore.document import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter

# 임베딩 모델 초기화
embedding_model = OpenAIEmbeddings(model="text-embedding-3-small")

print("임베딩 모델 준비 완료!")

### 도구 1: RAG 검색 도구

AI/데이터 관련 뉴스 기사를 벡터 DB에 저장하고 검색하는 도구입니다.

In [ ]:
# 샘플 뉴스 기사 데이터
news_articles = [
    {
        "title": "데이터브릭스, 차세대 AI 거버넌스 프레임워크 '유니티 카탈로그 2.0' 발표",
        "content": """
        2025년 1월, 데이터 및 AI 기업 데이터브릭스는 차세대 데이터 거버넌스 프레임워크인 '유니티 카탈로그 2.0'을 공개했다.
        이 프레임워크는 데이터, AI 모델, 관련 파이프라인 전체에 걸쳐 통합된 보안 및 거버넌스를 제공한다.
        특히, 생성형 AI 모델의 출력 결과에 대한 실시간 모니터링과 유해성 탐지 기능을 탑재했다.
        데이터브릭스의 CEO는 "AI의 민주화는 강력한 거버넌스 위에서만 가능하다"고 강조했다.
        """,
        "source": "데이터 이코노미",
        "date": "2025-01-20"
    },
    {
        "title": "AI 옵저버빌리티 플랫폼 '뉴럴와처', 2억 달러 투자 유치",
        "content": """
        AI 모델의 운영 상태를 실시간으로 감시하는 'AI 옵저버빌리티' 분야가 급성장하고 있다.
        스타트업 '뉴럴와처(NeuralWatcher)'는 시리즈 C 펀딩에서 2억 달러를 유치했다.
        뉴럴와처는 LLM 애플리케이션에서 발생하는 데이터 드리프트, 성능 저하, 환각(Hallucination) 현상을 자동 탐지한다.
        업계 전문가들은 AI 모델이 복잡해질수록 옵저버빌리티의 중요성은 더욱 커질 것이라고 전망했다.
        """,
        "source": "AI 스타트업 위클리",
        "date": "2025-01-22"
    },
    {
        "title": "구글 클라우드, '버텍스 AI 에이전트 빌더' 정식 출시",
        "content": """
        구글 클라우드는 버텍스 AI(Vertex AI)에 '에이전트 빌더' 기능을 정식으로 추가했다.
        코딩 경험이 없는 사용자도 자연어 명령어와 그래픽 인터페이스로 AI 에이전트를 구축할 수 있다.
        지식 베이스 연결, 구글 검색, 지도, 캘린더 등 다양한 도구를 추가할 수 있다.
        이는 '노코드 AI' 시대의 본격적인 시작을 의미한다.
        """,
        "source": "클라우드 인사이트",
        "date": "2025-01-25"
    }
]

print(f"준비된 뉴스 기사: {len(news_articles)}개")
for article in news_articles:
    print(f"  - {article['title'][:40]}...")

In [ ]:
# Document 객체로 변환
news_documents = [
    Document(
        page_content=article["content"].strip(),
        metadata={
            "title": article["title"],
            "source": article["source"],
            "date": article["date"]
        }
    )
    for article in news_articles
]

# 텍스트 분할
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50
)
split_docs = text_splitter.split_documents(news_documents)

# 벡터 DB 생성
news_vectordb = Chroma.from_documents(
    documents=split_docs,
    embedding=embedding_model,
    collection_name="news_for_agent"
)

# 리트리버 생성
retriever = news_vectordb.as_retriever(search_kwargs={"k": 3})

print(f"벡터 DB 생성 완료! ({len(split_docs)}개 청크)")

In [ ]:
from langchain.tools import Tool

# RAG 리트리버를 도구로 변환
def search_news(query: str) -> str:
    """뉴스 기사를 검색하여 관련 내용을 반환합니다."""
    docs = retriever.get_relevant_documents(query)
    if not docs:
        return "관련 뉴스를 찾지 못했습니다."
    
    results = []
    for doc in docs:
        results.append(f"[{doc.metadata.get('title', 'N/A')}]\n{doc.page_content}")
    return "\n\n".join(results)

news_search_tool = Tool(
    name="news_search",
    description="2025년 최신 AI 및 데이터 관련 뉴스 기사를 검색합니다. 유니티 카탈로그, AI 옵저버빌리티, 버텍스 AI 에이전트 빌더 관련 질문에 유용합니다.",
    func=search_news
)

print(f"도구 이름: {news_search_tool.name}")
print(f"도구 설명: {news_search_tool.description}")

### 도구 2: 웹 검색 도구

DuckDuckGo를 사용한 실시간 웹 검색 도구입니다.

In [ ]:
from langchain_community.tools import DuckDuckGoSearchRun

# 웹 검색 도구 생성
web_search_tool = DuckDuckGoSearchRun()

print(f"도구 이름: {web_search_tool.name}")
print(f"도구 설명: {web_search_tool.description}")

---

## 2.2 대화 메모리 통합

### 메모리의 필요성

```
메모리 없이:
User: 데이터브릭스가 뭐야?
Agent: 데이터브릭스는 데이터 및 AI 기업입니다.
User: 그 회사가 최근에 발표한 게 뭐야?  <- '그 회사'가 뭔지 모름!
Agent: 어떤 회사를 말씀하시는지...

메모리 있으면:
User: 데이터브릭스가 뭐야?
Agent: 데이터브릭스는 데이터 및 AI 기업입니다.
User: 그 회사가 최근에 발표한 게 뭐야?  <- '데이터브릭스'로 이해!
Agent: 데이터브릭스는 최근 유니티 카탈로그 2.0을 발표했습니다.
```

In [ ]:
from langchain.memory import ConversationBufferWindowMemory

# 최근 5개 대화를 기억하는 메모리
memory = ConversationBufferWindowMemory(
    k=5,
    memory_key="chat_history",
    return_messages=True
)

print("메모리 설정 완료!")
print(f"기억할 대화 수: {memory.k}개")

---

## 2.3 멀티-툴 에이전트 조립

### 최종 에이전트 구성

```
AI 뉴스 분석가 에이전트:
- 도구 1: news_search (RAG 기반 뉴스 검색)
- 도구 2: duckduckgo_search (웹 검색)
- 도구 3: get_current_time (현재 시간)
- 메모리: 최근 5개 대화 기억
```

In [ ]:
# 모든 도구 통합
all_tools = [news_search_tool, web_search_tool, get_current_time]

# 에이전트 프롬프트
agent_prompt = ChatPromptTemplate.from_messages([
    ("system", """
당신은 유능한 AI 뉴스 분석가입니다.
사용자의 질문에 따라 적절한 도구를 선택하여 답변하세요.

도구 사용 가이드:
- 유니티 카탈로그, AI 옵저버빌리티, 버텍스 AI 등 최신 AI 뉴스 관련 -> news_search 도구
- 실시간 정보(주가, 최신 뉴스, 일반 검색) -> duckduckgo_search 도구
- 현재 시간/날짜 -> get_current_time 도구

한국어로 친절하게 답변하세요.
"""),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad"),
])

# 에이전트 생성
multi_tool_agent = create_openai_tools_agent(llm, all_tools, agent_prompt)

# 에이전트 실행기 (메모리 포함)
multi_tool_executor = AgentExecutor(
    agent=multi_tool_agent,
    tools=all_tools,
    memory=memory,
    verbose=True,
    handle_parsing_errors=True  # 파싱 오류 자동 처리
)

print("멀티-툴 AI 뉴스 분석가 에이전트 준비 완료!")
print(f"사용 가능한 도구: {[t.name for t in all_tools]}")

---

## 2.4 에이전트 테스트

In [ ]:
# 테스트 1: RAG 도구 사용 (뉴스 검색)
question1 = "데이터브릭스가 발표한 유니티 카탈로그 2.0의 주요 특징이 뭐야?"
response1 = multi_tool_executor.invoke({"input": question1})

print("\n" + "=" * 60)
print(f"질문: {question1}")
print(f"\n답변: {response1['output']}")

In [ ]:
# 테스트 2: 웹 검색 도구 사용
question2 = "구글의 모회사 알파벳의 현재 주가는?"
response2 = multi_tool_executor.invoke({"input": question2})

print("\n" + "=" * 60)
print(f"질문: {question2}")
print(f"\n답변: {response2['output']}")

In [ ]:
# 테스트 3: 메모리 테스트 (이전 대화 기억)
question3 = "방금 첫 번째로 물어봤던 기술을 만든 회사가 어디였지?"
response3 = multi_tool_executor.invoke({"input": question3})

print("\n" + "=" * 60)
print(f"질문: {question3}")
print(f"\n답변: {response3['output']}")

In [ ]:
# 대화형 에이전트 함수
def ask_agent(question: str):
    """에이전트에게 질문하기"""
    print(f"\n질문: {question}")
    print("-" * 50)
    
    response = multi_tool_executor.invoke({"input": question})
    
    print(f"\n답변: {response['output']}")
    return response

# 테스트
ask_agent("AI 옵저버빌리티가 뭔지 설명해줘")

---

## 실습 퀴즈

**난이도**: (쉬움) ~ (어려움)

---

### Q1. LLM vs RAG vs 에이전트 비교하기

**문제**: 표준 LLM, RAG, AI 에이전트의 차이점을 각각 한 문장으로 설명하고, 각각에 적합한 사용 사례를 하나씩 제시하세요.

In [ ]:
# 여기에 답을 작성하세요


### Q2. ReAct 프레임워크 이해하기

**문제**: ReAct 프레임워크의 3단계(Thought, Action, Observation)를 설명하고, "내일 서울 날씨 알려줘"라는 질문에 대해 각 단계가 어떻게 진행될지 예시를 작성하세요.

In [ ]:
# 여기에 답을 작성하세요


### Q3. 도구 설명의 중요성

**문제**: 에이전트에서 도구의 `description`이 왜 중요한지 설명하고, 좋은 도구 설명의 3가지 조건을 서술하세요.

In [ ]:
# 여기에 답을 작성하세요


### Q4. 새로운 도구 만들기

**문제**: `@tool` 데코레이터를 사용하여 주어진 텍스트의 글자 수를 세는 `count_characters` 도구를 만들고 테스트하세요.

In [ ]:
# 여기에 코드를 작성하세요


### Q5. Agent Executor 이해하기

**문제**: `AgentExecutor`의 역할을 설명하고, `verbose=True` 옵션이 어떤 정보를 보여주는지 서술하세요.

In [ ]:
# 여기에 답을 작성하세요


### Q6. 에이전트 프롬프트 분석하기

**문제**: 본문의 에이전트 프롬프트에서 `agent_scratchpad`와 `chat_history`가 각각 어떤 역할을 하는지 설명하세요.

In [ ]:
# 여기에 답을 작성하세요


### Q7. 메모리 옵션 비교하기

**문제**: `ConversationBufferWindowMemory`의 `k` 파라미터가 의미하는 바를 설명하고, `k`를 너무 크게/작게 설정했을 때의 장단점을 서술하세요.

In [ ]:
# 여기에 답을 작성하세요


### Q8. 커스텀 에이전트 구축하기

**문제**: 다음 두 도구를 사용하는 에이전트를 만들고 테스트하세요.
1. `reverse_text`: 텍스트를 뒤집는 도구
2. `count_words`: 단어 수를 세는 도구

테스트 질문: "Hello World를 뒤집으면 뭐야?", "이 문장에 단어가 몇 개야: 파이썬은 재미있는 언어입니다"

In [ ]:
# 여기에 코드를 작성하세요


### Q9. 에이전트 디버깅

**문제**: 에이전트가 잘못된 도구를 선택하거나 예상과 다른 답변을 할 때, 문제를 진단하고 해결할 수 있는 방법 3가지를 서술하세요.

In [ ]:
# 여기에 답을 작성하세요


### Q10. 나만의 전문 에이전트 만들기

**문제**: 아래 주제 중 하나를 선택하여 최소 3개의 도구를 가진 에이전트를 구현하고, 3가지 질문으로 테스트하세요.

주제 선택:
1. 업무 비서 (일정 관리, 할일 추가, 시간 알림)
2. 데이터 분석가 (통계 계산, 문자열 처리, 검색)
3. 학습 도우미 (용어 설명, 문제 생성, 답변 평가)

In [ ]:
# 여기에 코드를 작성하세요


---

## 학습 정리

### Part 1: 기초 핵심 요약

| 개념 | 핵심 내용 | 실무 활용 |
|------|----------|----------|
| AI 에이전트 | LLM + 도구 + 추론 | 자율 작업 수행 AI |
| ReAct | Thought -> Action -> Observation | 단계적 문제 해결 |
| 도구 (Tool) | 외부 시스템과 상호작용 | API 호출, 검색 등 |
| @tool 데코레이터 | 함수를 도구로 변환 | 커스텀 도구 생성 |
| Agent Executor | ReAct 사이클 실행 | 에이전트 조율 |

### Part 2: 심화 핵심 요약

| 개념 | 핵심 내용 | 권장 설정 |
|------|----------|----------|
| 멀티-툴 에이전트 | 여러 도구 통합 | 도구 설명 명확히 |
| 메모리 | 대화 맥락 유지 | k=5~10 권장 |
| 도구 선택 | LLM이 설명 기반 결정 | 상세한 description |
| 에러 처리 | handle_parsing_errors | True 권장 |

### 에이전트 구현 체크리스트

```
1. 도구 준비:
   [ ] 필요한 도구 목록 정의
   [ ] @tool 데코레이터로 도구 생성
   [ ] 도구 설명(description) 명확히 작성
   [ ] 각 도구 개별 테스트
   
2. 에이전트 설정:
   [ ] LLM 선택 (temperature=0 권장)
   [ ] 프롬프트 템플릿 설계
   [ ] 메모리 설정 (필요 시)
   [ ] AgentExecutor 생성
   
3. 테스트:
   [ ] 각 도구별 질문 테스트
   [ ] 복합 질문 테스트
   [ ] 메모리(대화 맥락) 테스트
   [ ] 오류 상황 테스트
```

### 실무 팁

1. **도구 설명이 핵심**: LLM은 설명을 읽고 도구를 선택하므로 명확하게 작성
2. **verbose=True**: 개발 중에는 항상 켜서 추론 과정 확인
3. **temperature=0**: 에이전트는 일관된 행동이 중요하므로 낮게 설정
4. **에러 처리**: `handle_parsing_errors=True`로 안정성 확보
5. **도구 수 제한**: 너무 많은 도구는 선택 오류 증가 (5~10개 권장)